### 10.1. Dew point temperature calculation

This section explains the concepts and procedures related to dew point temperature calculations. While most experienced engineers are familiar with the phase envelope and general dew point principles, they may not be as familiar with topics like water saturation or recommended sampling procedures (very important for dew point calculation) unless they have deep experience in process simulation with a focus on vapor recovery. I strongly recommend you NOT to skip this section (except the first one if you alreay know it):

<ul style="margin-bottom: 1em;">
<li><p class="mt-0"><a class="internal-link" href="#10.1.1.-Phase-envelope-&-Dew-point-explained">10.1.1. Phase envelope & Dew point explained</a></p></li>
<li><p class="mt-0"><a class="internal-link" href="#10.1.2.-Water-Saturation">10.1.2. Water saturation</a></p></li>
<li><p class="mt-0"><a class="internal-link" href="#10.1.3.-Recommended-sampling-procedures">10.1.3. Recommended sampling procedures</a></p></li>
<li><p class="mt-0"><a class="internal-link" href="#10.1.4.-Process-simulation-modeling">10.1.4. Process simulation modeling</a></p></li>
</ul>

Note that dew point temperature calculation requires access to process simulation software. The one that I use is BR&E Promax.

### 10.1.1. Phase envelope & Dew point explained

To understand dew point, one must first understand a phase envelope—a graph that shows the phase (gas, liquid, two-phase, or supercritical) of a chemical mixture based on temperature and pressure. <a class="internal-link" href="#fig-69">Figure 69</a> shows the phase envelope of a 2500 Btu/scf vapor sample taken from an atmospheric tank for vapor recovery, with temperature on the x-axis and pressure on the y-axis.

The mixture's phase is determined by its position on the graph. If the (T, P) coordinate lies in the liquid region (green), the fluid is 100% liquid. In the gas region, it is 100% vapor. The supercritical region represents a supercritical fluid, which—for compressor operation purposes—behaves like a gas. Coordinates within the two-phase region indicate partial vapor and liquid. 

The dew point curve (red line) represents a set of (T, P) coordinates that form the boundary where the fluid is 100% vapor. Conversely, the bubble point curve (green line) marks the boundary where the fluid is 100% liquid. Within the two-phase region, the gray dashed lines indicate vapor fractions—for example, the 80% vapor line means the fluid is 80% vapor and 20% liquid. As the vapor fraction line approaches the bubble point curve, the vapor fraction decreases to 0%, becoming fully liquid. Likewise, as it nears the dew point curve, the vapor fraction approaches 100%.

<div class="row full_screen_margin_85 mobile_responsive_plot_full_width" id="fig-69">
    <div class="col"><img src="jupyter_images/screw_phase_envelope1.png"></div>
</div>
<div class="col-12 fig-title">
    <p class="image-description"><strong>Figre 69:</strong> Phase envelope of a hydrocarbon mixture compound. Sample is an upstream wellsite tank vapor with 2500 Btu/scf energy content. Simulated in BR&E Promax, and plotted in Python.</p>
</div>

<div class="solution_panel closed">
    <div class="solution_title">
        <p class="solution_title_string">Source Code For Figure (69)</p>
        <ul class="nav navbar-right panel_toolbox">
            <li><a class="collapse-link"><i class="fa fa-chevron-down"></i></a></li>
        </ul>
    <div class="clearfix"></div>
    </div>
    <div class="solution_content">
        <pre>
            <code class="language-python">
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

file = 'screw_compressor_article.xlsx'
dfs = [pd.read_excel(file, sheet_name=f'{n}% Vapor') for n in range(0, 101, 10)]

fig, ax = plt.subplots(figsize=(8, 4.5))

t_crit = 263.1727464076
p_crit = 1219.74099087

ax.plot(dfs[0]['Temperature'], dfs[0]['Pressure'], label='Bubble Point', color='green')
ax.plot(dfs[10]['Temperature'], dfs[10]['Pressure'], label='Dew Point', color='red')
ax.scatter(t_crit, p_crit, s=60, edgecolor='k', fc='white', zorder=3, label='Critical Point')

angles = [35, 45, 50, 57]  
for i, label, angle in zip([2, 4, 6, 8], ['20% Vapor', '40% Vapor', '60% Vapor', '80% Vapor'], angles):
    T = dfs[i]['Temperature'].values
    P = dfs[i]['Pressure'].values

    dT = np.diff(T)
    dP = np.diff(P)
    seg_lengths = np.sqrt(dT**2 + dP**2)
    cumulative = np.insert(np.cumsum(seg_lengths), 0, 0)
    total_len = cumulative[-1]

    half_len = total_len / 2
    idx = np.searchsorted(cumulative, half_len) - 1
    frac = (half_len - cumulative[idx]) / seg_lengths[idx]

    t_mid = T[idx] + frac * (T[idx + 1] - T[idx])
    p_mid = P[idx] + frac * (P[idx + 1] - P[idx])

    ax.plot(T, P, color='grey', ls='--', lw=1, alpha=0.99)
    ax.text(t_mid, p_mid, label, fontsize=9, rotation=angle,
            ha='center', va='center', color='grey',
            bbox=dict(facecolor='white', edgecolor='none', pad=1.5, alpha=1))


ax.set_ylim(0, 1600)
ax.set_xlim(-150, 400)
x_min, x_max = ax.get_xlim()
y_max = ax.get_ylim()[1]

# Supercritical region
ax.plot([t_crit, t_crit], [p_crit, y_max], color='black')
ax.plot([t_crit, x_max], [p_crit, p_crit], color='black')
ax.fill_between([t_crit, x_max], p_crit, y_max, color='black', alpha=0.05, zorder=0)
ax.text(289, 1370, 'Supercritical', fontsize=12, ha='left', color='black', alpha=0.5)

# Gas region
T = dfs[10]['Temperature']
P = dfs[10]['Pressure']
t_interp = np.interp(p_crit, P, T)
verts = [(t, p) for t, p in zip(T, P) if p <= p_crit]
verts += [(t_interp, p_crit), (x_max, p_crit), (x_max, P.min()), (T.iloc[0], P.iloc[0])]
ax.add_patch(Polygon(verts, closed=True, color='red', alpha=0.05, zorder=0))
ax.text(330, 470, 'Gas', fontsize=12, ha='left', color='red', alpha=0.99)

# Liquid region
T = dfs[0]['Temperature']
P = dfs[0]['Pressure']
mask = T <= t_crit
T_sel, P_sel = T[mask], P[mask]
verts = [(x_min, y_max), (x_min, P_sel.iloc[0])] + list(zip(T_sel, P_sel)) + [(T_sel.iloc[-1], y_max)]
ax.add_patch(Polygon(verts, closed=True, color='green', alpha=0.05, zorder=0))
ax.text(-80, 870, 'Liquid', fontsize=12, ha='left', color='green', alpha=0.99)

ax.text(150, 900, '2-Phase Region', fontsize=12, ha='left', color='black', alpha=0.99,
       bbox=dict(facecolor='white', edgecolor='none', pad=1.5, alpha=1))

ax.set_xlabel('Temperature [°F]')
ax.set_ylabel('Pressure [psig]')
ax.grid(True)
ax.minorticks_on()
ax.grid(axis='both', which='minor', color='grey', linestyle='--', alpha=0.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, ncol=3)

ax.text(0.98, 0.05, 'aegis4048.github.io', fontsize=10, ha='right', transform=ax.transAxes, color='grey', alpha=0.5)

def setbold(txt): return ' '.join([r"$\bf{" + item + "}$" for item in txt.split(' ')])
fig.suptitle(setbold('Phase Envelope') + ', 2500 Btu/scf Tank Vapor', verticalalignment='top',
             x=0, horizontalalignment='left', fontsize=11)
yloc = 0.9
ax.annotate('', xy=(0.01, yloc + 0.01), xycoords='figure fraction',
            xytext=(1.02, yloc + 0.01), arrowprops=dict(arrowstyle="-", color='k', lw=0.7))

fig.tight_layout()
            </code>
        </pre>
    </div>
</div>

<hr>

For oil-flooded screw compressors, the operating pressure and temperature must stay within the gas region to prevent condensation and lube oil contamination. The operating point should not cross into the two-phase region. <a class="internal-link" href="#fig-70">Figure 70</a> shows a zoomed-in phase envelope of the same sample above, truncated at 400 psig on the y-axis, noting that typical screw compressor discharge pressure limits are at 350 psig. With a 230°F thermostat valve setpoint (which defines the compressor's operating temperature—see <a class="internal-link" href="#7.4.-3-Way-thermostatic-valve">Section 7.4</a>), condensation risk begins around 150 psig, where the dew point reaches 202°F. Since 15–20°F above dew point is recommended, and 30°F is ideal, this leaves just about enough margin.

At 200 psig, the dew point rises to 218°F, leaving only a 12°F buffer—below the ideal range. At 250 psig and above, condensation becomes unavoidable, leading to lube oil contamination. <strong><u>The higher the discharge pressure, the higher the risk of oil contamination—depending on gas composition.</u></strong>

If the salesline requirement is for this tank vapor recovery application is below 150-180 psig, oil-flooded screw compressors remain suitable. However, for higher-pressure applications, alternatives like rotary vane compressors—which do not suffer from lube oil contamination—should be considered. Note that rotary vane units are generally more expensive than screws.

<div class="row full_screen_margin_85 mobile_responsive_plot_full_width" id="fig-70">
    <div class="col"><img src="jupyter_images/screw_phase_envelope2.png"></div>
</div>
<div class="col-12 fig-title">
    <p class="image-description"><strong>Figre 70:</strong> Zoomed-in version of <a class="internal-link" href="#fig-69">Figure 69</a>. Simulated in BR&E Promax, and plotted in Python. Oil-flooded screw compressors typically reach up to 350 psig. The plot shows dew point temperatures at various pressures. As pressure increases, dew point temperature rises. With a 230°F thermostat setpoint and a recommended 30°F buffer, condensation risk begins above 150 psig. 
</p>
</div>

<div class="solution_panel closed">
    <div class="solution_title">
        <p class="solution_title_string">Source Code For Figure (70)</p>
        <ul class="nav navbar-right panel_toolbox">
            <li><a class="collapse-link"><i class="fa fa-chevron-down"></i></a></li>
        </ul>
    <div class="clearfix"></div>
    </div>
    <div class="solution_content">
        <pre>
            <code class="language-python">
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

file = 'screw_compressor_article.xlsx'
dfs = [pd.read_excel(file, sheet_name=f'{n}% Vapor') for n in range(0, 101, 10)]

fig, ax = plt.subplots(figsize=(8, 4.5))

t_crit = 263.1727464076
p_crit = 1219.74099087

ax.plot(dfs[0]['Temperature'], dfs[0]['Pressure'], label='Bubble Point', color='green')
ax.plot(dfs[10]['Temperature'], dfs[10]['Pressure'], label='Dew Point', color='red')

ax.set_ylim(0, 400)
ax.set_xlim(-150, 400)
x_min, x_max = ax.get_xlim()
y_min, y_max = ax.get_ylim()

angles = [60, 60, 60, 60]
for i, label, angle in zip([2, 4, 6, 8], ['20% Vapor', '40% Vapor', '60% Vapor', '80% Vapor'], angles):
    T = dfs[i]['Temperature'].values
    P = dfs[i]['Pressure'].values

    # Clip to plotting bounds
    mask = (T >= x_min) & (T <= x_max) & (P >= y_min) & (P <= y_max)
    T_clipped = T[mask]
    P_clipped = P[mask]

    if len(T_clipped) < 2:
        continue  # Skip if not enough points to compute a midpoint

    dT = np.diff(T_clipped)
    dP = np.diff(P_clipped)
    seg_lengths = np.sqrt(dT**2 + dP**2)
    cumulative = np.insert(np.cumsum(seg_lengths), 0, 0)
    total_len = cumulative[-1]

    half_len = total_len / 2
    idx = np.searchsorted(cumulative, half_len) - 1
    frac = (half_len - cumulative[idx]) / seg_lengths[idx]

    t_mid = T_clipped[idx] + frac * (T_clipped[idx + 1] - T_clipped[idx])
    p_mid = P_clipped[idx] + frac * (P_clipped[idx + 1] - P_clipped[idx])

    ax.plot(T, P, color='grey', ls='--', lw=1, alpha=0.99)
    ax.text(t_mid, p_mid, label, fontsize=9, rotation=angle,
            ha='center', va='center', color='grey',
            bbox=dict(facecolor='white', edgecolor='none', pad=1.5, alpha=1))

# Gas region
T = dfs[10]['Temperature']
P = dfs[10]['Pressure']
t_interp = np.interp(p_crit, P, T)
verts = [(t, p) for t, p in zip(T, P) if p <= p_crit]
verts += [(t_interp, p_crit), (x_max, p_crit), (x_max, P.min()), (T.iloc[0], P.iloc[0])]
ax.add_patch(Polygon(verts, closed=True, color='red', alpha=0.05, zorder=0))
ax.text(330, 350, 'Gas', fontsize=12, ha='left', color='red', alpha=0.99)

# Liquid region
T = dfs[0]['Temperature']
P = dfs[0]['Pressure']
mask = T <= t_crit
T_sel, P_sel = T[mask], P[mask]
verts = [(x_min, y_max), (x_min, P_sel.iloc[0])] + list(zip(T_sel, P_sel)) + [(T_sel.iloc[-1], y_max)]
ax.add_patch(Polygon(verts, closed=True, color='green', alpha=0.05, zorder=0))
ax.text(-130, 350, 'Liquid', fontsize=12, ha='left', color='green', alpha=0.99)

ax.text(20, 290, '2-Phase Region', fontsize=12, ha='left', color='black', alpha=0.99,
       bbox=dict(facecolor='white', edgecolor='none', pad=1.5, alpha=1))

ax.set_xlabel('Temperature [°F]')
ax.set_ylabel('Pressure [psig]')
ax.grid(True)
ax.minorticks_on()
ax.grid(axis='both', which='minor', color='grey', linestyle='--', alpha=0.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.axvline(x=230, color='k', linestyle='-', linewidth=2, label='230°F Thermostat Setpoint')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, ncol=3, loc='lower right')


ax.scatter(181.8355342547, 100, marker='D', s=40, fc='yellow', edgecolor='purple', zorder=10)
ax.text(195, 100, 'Dew T=182°F @ 100 psig', fontsize=10, ha='left', va='center', color='purple')

ax.scatter(202.3073722246, 150, marker='D', s=40, fc='yellow', edgecolor='purple', zorder=10)
ax.text(217, 150, 'Dew T=202°F @ 150 psig', fontsize=10, ha='left', va='center', color='purple')

ax.scatter(217.8299782129, 200, marker='D', s=40, fc='yellow', edgecolor='purple', zorder=10)
ax.text(230, 200, 'Dew T=218°F @ 200 psig', fontsize=10, ha='left', va='center', color='purple')

ax.scatter(230.2676365518, 250, marker='D', s=40, fc='yellow', edgecolor='purple', zorder=10)
ax.text(247, 250, 'Dew T=230°F @ 250 psig', fontsize=10, ha='left', va='center', color='purple')

ax.scatter(240.5520855614, 300, marker='D', s=40, fc='yellow', edgecolor='purple', zorder=10)
ax.text(257, 300, 'Dew T=241°F @ 300 psig', fontsize=10, ha='left', va='center', color='purple')

ax.text(0.98, 0.14, 'aegis4048.github.io', fontsize=10, ha='right', transform=ax.transAxes, color='grey', alpha=0.5)

def setbold(txt): return ' '.join([r"$\bf{" + item + "}$" for item in txt.split(' ')])
fig.suptitle(setbold('Phase Envelope') + ', 2500 Btu/scf Tank Vapor (zoomed-in)', verticalalignment='top',
             x=0, horizontalalignment='left', fontsize=11)
yloc = 0.9
ax.annotate('', xy=(0.01, yloc + 0.01), xycoords='figure fraction',
            xytext=(1.02, yloc + 0.01), arrowprops=dict(arrowstyle="-", color='k', lw=0.7))

fig.tight_layout()
            </code>
        </pre>
    </div>
</div>

<hr>

Lube oil contamination problems tend to be a challenge for high-BTU applications above 2000 Btu/scf. These are usually low pressure vapor recovery applications for either VRTs operating 1-5 psig, or tanks operating below 1 psig. The issue arises because higher-BTU gases exhibit higher dew point temperatures at a given pressure, increasing the risk of process gas condensation into liquid during compression.

<a class="internal-link" href="#fig-71">Figure 71</a> shows phase envelopes for three representative gas samples: 2000, 2500, and 3000 Btu/scf. These values reflect realistic conditions in VRT and tank vapor recovery systems. As Btu content increases, the phase envelope shifts rightward—indicating higher dew point temperatures at equal pressures. This means that as samples get richer (high Btu), compressors must operate at higher temperatures to stay in the gas region.



The three intersection points on the 200 psig isobar represent dew point temperatures for each sample:

<ul style="margin-bottom: 1em;">
<li><p class="mt-0">2000 Btu/scf → 172°F</p></li>
<li><p class="mt-0">2500 Btu/scf → 218°F</p></li>
<li><p class="mt-0">3000 Btu/scf → 249°F</p></li>
</ul>

The vertical line at 230°F reflects the typical thermostat valve setpoint for oil-flooded screw compressors, which maintains the normal operating temperature of the unit (see <a class="internal-link" href="#7.4.-3-Way-thermostatic-valve">Section 7.4</a>). At 200 psig, both the 2000 and 2500 Btu/scf gases remain below this threshold, though the 2500 sample approaches the limit with minimal safety margin (you want at least 15-30°F margin). In contrast, the dew point temperature for the 3000 Btu/scf gas 249°F exceeds the 230°F setpoint, guaranteeing condensation and subsequent lube oil contamination.

For this reason, screw compressors are not feasible for this 3000 Btu/scf gas at 200 psig discharge (potentially not feasible for 3000 Btu/scf gas as well due to minimal operating temperature margin). Alternatives like rotary vane compressors—which do not rely on lube oil—should be considered. While more expensive, rotary vanes are better suited for high-BTU recovery. The 2000 Btu/scf gas, however, provides enough margin to safely operate within the gas region across the full pressure range of typical screw compressors (60–350 psig).

<strong><u>The higher the gas energy content (Btu/scf), the higher the risk of oil contamination—depending on discharge pressure.</u></strong> If the application of your interest is above 2000 Btu/scf, it is highly recommended to run a dew point check when considering an oil-flooded screw compressor installation (send me an email inquiry, and I can run one for you if its for business).

It's important to note that phase envelopes can vary significantly between gas samples, even if they have similar Btu/scf values. For example, while the 2500 Btu/scf sample discussed here shows a 218°F dew point at 200 psig, another sample with the same energy content and pressure could show a much lower dew point—such as 180°F. I’ve observed this firsthand in the field on multiple occasions. Very common.


<div class="row full_screen_margin_85 mobile_responsive_plot_full_width" id="fig-71">
    <div class="col"><img src="jupyter_images/screw_phase_envelope3.png"></div>
</div>
<div class="col-12 fig-title">
    <p class="image-description"><strong>Figre 71:</strong> Phase envelopes of various hydrocarbon mixtures at different energy contents (Btu/scf). Simulated in BR&E Promax, and plotted in Python. The Btu/scf samples have higher dew point temperatures at the same 200# discharge pressure. The 230°F vertical line represents the typical 230°F setpoint for a 3-way thermostat valve that determines the normal operating temperature of an oil-flooded screw compressor. The dew point must be below this 230°F limit to avoid condensation and subsequent lube oil contamination issues.
</p>
</div>

<div class="solution_panel closed">
    <div class="solution_title">
        <p class="solution_title_string">Source Code For Figure (71)</p>
        <ul class="nav navbar-right panel_toolbox">
            <li><a class="collapse-link"><i class="fa fa-chevron-down"></i></a></li>
        </ul>
    <div class="clearfix"></div>
    </div>
    <div class="solution_content">
        <pre>
            <code class="language-python">
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

file = 'screw_compressor_article.xlsx'
dfs = [pd.read_excel(file, sheet_name=f'{n}% Vapor') for n in range(0, 101, 10)]
dfs.append(pd.read_excel(file, sheet_name='2000 Btu Bubble'))
dfs.append(pd.read_excel(file, sheet_name='2000 Btu Dew'))
dfs.append(pd.read_excel(file, sheet_name='3000 Btu Bubble'))
dfs.append(pd.read_excel(file, sheet_name='3000 Btu Dew'))

fig, ax = plt.subplots(figsize=(8, 4.5))

t_crit1 = 263.1727464076 # 2500 Btu/scf
p_crit1 = 1219.74099087  # 2500 Btu/scf

t_crit2 = 182.7752706    # 2000 Btu/scf
p_crit2 = 1552.581899    # 2000 Btu/scf

t_crit3 = 312.9541125064 # 3000 Btu/scf
p_crit3 = 877.4475701297 # 3000 Btu/scf

ax.plot(dfs[0]['Temperature'], dfs[0]['Pressure'], label='Bubble Point', color='green')
ax.plot(dfs[10]['Temperature'], dfs[10]['Pressure'], label='Dew Point', color='red')
ax.scatter(t_crit1, p_crit1, s=60, edgecolor='k', fc='white', zorder=3, label='Critical Point')

ax.plot(dfs[11]['Temperature'], dfs[11]['Pressure'], color='green', ls='--')
ax.plot(dfs[12]['Temperature'], dfs[12]['Pressure'], color='red', ls='--')
ax.scatter(t_crit2, p_crit2, s=60, edgecolor='k', fc='white', zorder=3, ls='--')

ax.plot(dfs[13]['Temperature'], dfs[13]['Pressure'], color='green', ls='dotted')
ax.plot(dfs[14]['Temperature'], dfs[14]['Pressure'], color='red', ls='dotted')
ax.scatter(t_crit3, p_crit3, s=60, edgecolor='k', fc='white', zorder=3, ls='dotted')

ax.text(10, 1010, '2000 Btu/scf', fontsize=9, rotation=45, color='grey',
        bbox=dict(facecolor='white', edgecolor='none', pad=1.5, alpha=1))
ax.text(10, 630, '2500 Btu/scf', fontsize=9, rotation=35, color='grey',
        bbox=dict(facecolor='white', edgecolor='none', pad=1.5, alpha=1))
ax.text(10, 260, '3000 Btu/scf', fontsize=9, rotation=19, color='grey',
        bbox=dict(facecolor='white', edgecolor='none', pad=1.5, alpha=1))

ax.set_ylim(0, 1600)
ax.set_xlim(-150, 400)
x_min, x_max = ax.get_xlim()
y_max = ax.get_ylim()[1]

ax.set_xlabel('Temperature [°F]')
ax.set_ylabel('Pressure [psig]')
ax.grid(True)
ax.minorticks_on()
ax.grid(axis='both', which='minor', color='grey', linestyle='--', alpha=0.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, ncol=1, loc='upper left')

ax.axvline(x=230, color='k', linestyle='-', linewidth=2, label='230°F Thermostat Setpoint')
ax.text(237, 1080, '230°F Setpoint', rotation=90, va='bottom', ha='left', fontsize=10, color='black')

ax.axhline(y=200, color='k', linestyle='-', linewidth=2, label='230°F Thermostat Setpoint')
ax.text(400, 220, '200# Discharge', va='bottom', ha='right', fontsize=10, color='black')

ax.text(0.02, 0.62, 'aegis4048.github.io', fontsize=10, ha='left', transform=ax.transAxes, color='grey', alpha=0.5)

ax.scatter(249.0971642597, 200, marker='D', s=40, fc='yellow', edgecolor='purple', zorder=10)
ax.text(249.0971642597, 200 + 70, '249°F', fontsize=10, ha='left', va='center', color='purple')

ax.scatter(217.8299958163, 200, marker='D', s=40, fc='yellow', edgecolor='purple', zorder=10)
ax.text(217.8299958163, 200 + 70, '218°F', fontsize=10, ha='center', va='center', color='purple')

ax.scatter(172.7152181272, 200, marker='D', s=40, fc='yellow', edgecolor='purple', zorder=10)
ax.text(172.7152181272, 200 + 70, '172°F', fontsize=10, ha='right', va='center', color='purple')

def setbold(txt): return ' '.join([r"$\bf{" + item + "}$" for item in txt.split(' ')])
fig.suptitle(setbold('Phase Envelope') + ', 3000 vs. 2500 vs. 2000 Btu/scf samples', verticalalignment='top',
             x=0, horizontalalignment='left', fontsize=11)
yloc = 0.9
ax.annotate('', xy=(0.01, yloc + 0.01), xycoords='figure fraction',
            xytext=(1.02, yloc + 0.01), arrowprops=dict(arrowstyle="-", color='k', lw=0.7))

fig.tight_layout()
            </code>
        </pre>
    </div>
</div>

<div class="highlights red-theme" id="warning-regional">
<div class="highlights-title red-theme">WARNING: Impact of Water Saturation on Phase Envelope</div>
<div class="highlights-content red-theme">
        <p>Phase envelopes are intended for two-phase systems. In oil and gas, water introduces a third phase because it is immiscible with hydrocarbons. <a class="internal-link" href="#fig-D">Figure D</a> demonstrates <strong><u>distortion on the dew point curve near the critical point due to presence of water</u></strong>. Adding just 2% water distorts the envelope. This distortion is common in low-pressure vapor recovery systems, which often have 0–8% water content depending on temperature.</p>
    <p>ProMax (the simulator used here) excludes water when constructing phase envelopes by default. This can lead to incorrect dew point readings if water is present. <a class="internal-link" href="#table-5">Table 5</a> shows the impact of water fractions on increasing dew point temperatures. Fortunately, Promax has a different dew point calculation method that accounts for water content. Dont'd read dew points off the phase envelopes. Instead, use direct dew point calculation methods that account for water (In Promax, <i>"Vapor Pressure, Dew, Bubble Point"</i> tab in <i>"Add Analysis"</i> supports this). If using a different simulator, confirm how your software vendor how it handles water inclusion for dew point calculations.</p>
    <div class="row full_screen_margin_85 mobile_responsive_plot_full_width" id="fig-D">
<div class="col"><img src="jupyter_images/screw_phase_envelope4.png" style="border: 1px solid #ddd;"></div>
</div>
<div class="col-12 fig-title"><p class="image-description"><strong>Figure D:</strong> Phase envelope of the same 2000 Btu/scf gas sample shown in <a class="internal-link" href="#fig-71">Figure 71</a>, with 2% saturated water added to reflect field conditions—where 0–8% water content is typical in low-pressure vapor recovery. The envelope shows distortion near the critical point on the dew point curve. This occurs because a 2D phase envelope assumes a two-phase system, which breaks down when water is present, as water is immiscible with hydrocarbons.</p></div>
        <div class="solution_panel closed" style="margin-top: 7px;">
        <div class="solution_title solution_warning">
            <p class="solution_title_string">Source Code For The Figure (D)</p>
            <ul class="nav navbar-right panel_toolbox">
                <li><a class="collapse-link"><i class="fa fa-chevron-down"></i></a></li>
            </ul>
            <div class="clearfix"></div>
        </div>
        <div class="solution_content">
            <pre>
                <code class="language-python">
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

file = 'screw_compressor_article.xlsx'
dfs = []
dfs.append(pd.read_excel(file, sheet_name='2% Water Bubble'))
dfs.append(pd.read_excel(file, sheet_name='2% Water Dew'))


fig, ax = plt.subplots(figsize=(8, 4.5))


t_crit1 = 180.4400727908 # 2500 Btu/scf
p_crit1 = 1563.074498018  # 2500 Btu/scf


ax.plot(dfs[0]['Temperature'], dfs[0]['Pressure'], label='Bubble Point', color='green')
ax.plot(dfs[1]['Temperature'], dfs[1]['Pressure'], label='Dew Point', color='red')
ax.scatter(t_crit1, p_crit1, s=60, edgecolor='k', fc='white', zorder=3, label='Critical Point')


ax.set_ylim(0, 2000)
ax.set_xlim(None, 400)


ax.set_xlabel('Temperature [°F]')
ax.set_ylabel('Pressure [psig]')
ax.grid(True)
ax.minorticks_on()
ax.grid(axis='both', which='minor', color='grey', linestyle='--', alpha=0.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, ncol=1, loc='upper left')


ax.text(0.02, 0.62, 'aegis4048.github.io', fontsize=10, ha='left', transform=ax.transAxes, color='grey', alpha=0.5)


ax.text(0.05, 0.37, '2% Mol frac water', fontsize=14, ha='left', transform=ax.transAxes, color='blue', fontweight='bold')
ax.text(0.05, 0.3, 'Presence of water fraction distorts 2D phase envelope', fontsize=10, ha='left', transform=ax.transAxes, color='black')
ax.text(0.05, 0.245, 'for the dew point curve near a critical point', fontsize=10, ha='left', transform=ax.transAxes, color='black')


def setbold(txt): return ' '.join([r"$\bf{" + item + "}$" for item in txt.split(' ')])
fig.suptitle(setbold('Phase Envelope') + ', Impact of water fraction on phase envelope', verticalalignment='top',
             x=0, horizontalalignment='left', fontsize=11)
yloc = 0.9
ax.annotate('', xy=(0.01, yloc + 0.01), xycoords='figure fraction',
            xytext=(1.02, yloc + 0.01), arrowprops=dict(arrowstyle="-", color='k', lw=0.7))


fig.tight_layout()
                </code>
            </pre>
        </div>
    </div>
    </div>
</div>

### 10.1.2. Water Saturation

Water saturation in a gas stream increases the dew point temperature at a given pressure. The severity of saturation depends on both the source pressure and temperature. Water saturation is rarely an issue for gas from vessels operating above 50 psig but becomes critical in low-pressure applications such as tank or VRT vapor recovery—especially when compressor inlet temperatures exceed 110°F. Higher temperatures increase the amount of water vapor a stream can hold before dropping liquid.

A major challenge in dew point calculations is that most laboratory gas analyses exclude water content unless specifically requested. Standard reports are typically "dry" analyses without water. Based on my experiences, water fractions in low-pressure gas streams commonly range from 0–8%, depending on temperature. Because of this, I often estimate water content using a worst-case assumption of 100% water saturation, which generally overestimates the dew point. To avoid this uncertainty, operators should always request that water content be included in lab reports.

The following sections provide additional detail:

<ul style="margin-bottom: 1em;">
<li><p class="mt-0"><a class="internal-link" href="#water_sat_0">Definition of water saturation</a></p></li>
<li><p class="mt-0"><a class="internal-link" href="#water_sat_1">Impact of water saturation on dew point temperature</a></p></li>
<li><p class="mt-0"><a class="internal-link" href="#water_sat_2">Impact of temperature and pressure on water saturation</a></p></li>
<li><p class="mt-0"><a class="internal-link" href="#water_sat_3">Problems with standard lab analysis omitting water fractions</a></p></li>
<li><p class="mt-0"><a class="internal-link" href="#water_sat_4">Uncertainties in predicting water saturation</a></p></li>
</ul>

<hr>

<div id="water_sat_0"></div>

**Definition of water saturation**

Water saturation refers to the amount of water vapor a gas stream holds relative to the maximum it can contain without forming the first drop of liquid at a given temperature and pressure. In upstream systems, gas that has been in contact with bulk free water—such as in separators or heater treaters—is typically fully saturated. In contrast, gas from vessels like VRTs or tanks may or may not be fully saturated, because flash gas separates immediately upon entry and the only opportunity for it to contact any entrained water is during the short pipe run between the upstream separator and the vessel. If this pipe length is too short, the gas may not reach equilibrium with entrained water and can exit less than fully saturated (this depends on water entraintment, typically 0-2%). This matters because increase in water vapor increases dew point temperature. Accurately estimating water saturation is critical in compression system design, but is often difficult in practice since standard lab analyses do not include water content unless explicitly requested.

<hr>

<div id="water_sat_1"></div>

**Impact of water saturation on dew point temperature**

Presence of saturated water in a gas stream increases the dew point temperature at a given pressure. This raises the risk of condensation and lube oil contamination in oil-flooded screw compressors.  Since screw compressors typically operate around 230°F, maintaining a safe margin below this temperature is critical.

<a class="internal-link" href="#table-5">Table 5</a> shows process simulation results using the same 2,000 Btu/scf dry gas sample from <a class="internal-link" href="#fig-71">Figure 71</a>. The simulation artificially added water saturation to reach specified water mol fraction levels without dropping liquid. The simulation model used to generate these results is shown in <a class="internal-link" href="#fig-72">Figure 72</a>.

At a constant discharge pressure of 200 psig for the 2,000 Btu/scf gas sample, the dew point temperaturs were:

<ul style="margin-bottom: 1em;">
<li><p class="mt-0">0% water mol frac → 172°F</p></li>
<li><p class="mt-0">5% water mol frac → 196°F</p></li>
<li><p class="mt-0">7% water mol frac → 212°F</p></li>
<li><p class="mt-0">10% water mol frac → 231°F</p></li>
</ul>

Compared to the dry gas case at 172°F, the dew point rises significantly—by 40°F at 7% water content. This brings the dew point close to the 230°F compressor operating limit, leaving little room for a desirable margin of 15–30°F below that limit. At 10% water, the dew point reaches 231°F, exceeding the max operating range and guaranteeing liquid dropout and lube oil contamination.

In the results table, the dew point increases are minor up to about 3% water mol fraction, but above this threshold, increases become substantial—typically 4 to 6°F for each additional 1% of water added. Note that these are highly specific to sample compositions and should not be generalized for all samples.

<div class="row" id="table-5">
    <div class="col"><img src="jupyter_images/screw_watersat2.png"></div>
</div>
<div class="col-12 fig-title">
    <p class="image-description"><strong>Table 5:</strong> Dew point temperature as a function of water mol fraction and discharge pressure, based on the same 2,000 Btu/scf gas stream used in <a class="internal-link" href="#fig-71">Figure 71</a>. Simulations were performed using BR&E ProMax, assuming saturated vapor at 150°F tank conditions. High tank temperature is used specifically to allow maximum water saturation as possible. A saturator block is used to artificially increase water saturation from the original sample. This table is constructed from the simulation model shown in <a class="internal-link" href="#fig-72">Figure 72</a>.</p>
</div>

<div class="row" id="fig-72">
    <div class="col"><img src="jupyter_images/screw_watersat5.png"></div>
</div>
<div class="col-12 fig-title">
    <p class="image-description"><strong>Figure 72:</strong> Promax simulation model used to generate results in <a class="internal-link" href="#table-5">Table 5</a>. Low- and high-BTU gas samples are blended to achieve a target heating value of 2,000 Btu/scf. The simulation predicts a dew point temperature of 196°F for tank vapor recovery at 200 psig discharge pressure and 5% water mol fraction. Inlet temperature is intentionally set at 150°F—higher than typical tank conditions (80–130°F)—to allow for greater water vapor capacity without dropping liquid. At this temperature, 5% mol fraction corresponds to 15.47% of full saturation, while full water saturation reaches 25.4% water mol fraction (not shown). Note that the heating value decreased post-saturation (2000 Btu/scf → 1902.5 Btu/scf), because water has no energy content. The DVDR-1 and MIX-101 blocks are just convenience components to allow targeting a specified water mol fraction with a Promax solver.</p>
</div>

<hr>

<div id="water_sat_2"></div>

**Impact of temperature and pressure on water saturation**

Increase in temperature increases 

<div class="row" id="table-5">
    <div class="col"><img src="jupyter_images/screw_watersat1.png"></div>
</div>
<div class="col-12 fig-title">
    <p class="image-description"><strong>Table 6:</strong> Test</p>
</div>

<hr>

<div id="water_sat_3"></div>

**Problems with standard lab analysis omitting water fractions**

<hr>

<div id="water_sat_4"></div>

**Uncertainties in predicting water saturation**

Water saturation refers to how much water vapor is present in a gas stream based on whether the gas has been in direct contact with bulk water. If the gas has interacted with bulk free water—like in a primary separator or heater treater—it becomes <u>fully saturated</u>, meaning it holds as much water as it can in vapor form without forming liquid. If the gas comes from vessels with minimal free water, like VRTs or tanks, it <u>may not or may not be fully saturated</u>, and could still absorb more water before reaching that point. This vapor-phase water cannot be removed by gravity-based separators like knockouts or scrubbers, as it exists as part of the gas phase and requires specialized dehydration equipment, such as glycol units. 

To illustrate this, Promax saturator blocks are used to artificially increase water content until just below the point of free-water dropout:

<ul style="margin-bottom: 1em;">
<li><p class="mt-0"><a class="internal-link" href="#fig-72">Figure 72</a>: Simulates a 0% entrained water case. The heater treater gas is already fully saturated due to direct contact with bulk water. The water fraction remains constant at 4.52% before and after the saturator. In contrast, the tank gas—initially at 6.69%—increases to 11.2% after saturation, indicating partial saturation of the original flash gas.</p></li>
<li><p class="mt-0"><a class="internal-link" href="#fig-73">Figure 73</a>: Same simulation setup, but with 1% water entrained in the oil. This small volume is sufficient to fully saturate the tank vapor under equilibrium conditions. The water fraction remains unchanged at 11.2% before and after the saturator block, confirming full saturation.</p></li>
</ul>

Whether small amounts of entrained water (typically 0-2%) are enough to fully saturate the gas depends on specific conditions. Flash gas always separates immediately upon entering downstream vessels like VRTs or tanks, which means the only opportunity for contact with entrained water is during the short pipe run between the upstream separator and the vessel. If this pipe length does not provide enough residence time, the gas may not reach equilibrium and can exit less than fully saturated. This behavior is not captured by steady-state simulators like Promax, which assume perfect equilibrium. Field gas samples I’ve reviewed that explicitly report water fractions often show values below full saturation, even though simulations suggest that as little as 1 percent entrained water can fully saturate the vapor stream.

The uncertainty around water saturation largely stems from the fact that standard lab gas analyses do not include water vapor content unless explicitly requested (see <a class="internal-link" href="#water_sat_3">below</a>). Without this data, engineers are forced to estimate or assume saturation levels—often defaulting to 100% as a conservative approach. While safe, this assumption introduces unnecessary uncertainty in dew point predictions and can lead to overly cautious compressor selection decisions.


<div class="row" id="fig-72">
    <div class="col"><img src="jupyter_images/screw_watersat3.png"></div>
</div>
<div class="col-12 fig-title">
    <p class="image-description"><strong>Figure 72:</strong> Process simulation in BR&E ProMax showing water saturation differences between gas streams from a vessel containing bulk water (heater treater) and one with minimal water content (oil tank). Saturator blocks ("SAT-1" and "SAT-2") raise the water content until just below free-water dropout. For the heater treater gas, the water fraction remains unchanged at 4.52%, confirming it’s already fully saturated. The tank gas increases from 6.69% to 11.2%, indicating it wasn’t fully saturated initially due to limited contact with bulk water. <i>Note: saturator blocks are simulation tools only and do not exist in real-world systems.</i></p>
</div>

<div class="row" id="fig-73">
    <div class="col"><img src="jupyter_images/screw_watersat4.png"></div>
</div>
<div class="col-12 fig-title">
    <p class="image-description"><strong>Figure 73:</strong> Same simulation setup as <a class="internal-link" href="#fig-72">Figure 72</a>, but with 1% free water entrained in the oil entering the tank. Unlike Figure 72, the tank gas in this case is already fully saturated upon entry, as indicated by the unchanged water fraction of 11.2% before and after the saturator block ("SAT-2"). This demonstrates that even small volumes of entrained water can be sufficient to fully saturate the flash gas.</p>
</div>

**Uncertainties in predicting water saturation**

The uncertainty of whether a gas stream is fully or partially saturated becomes important because water saturation increased dew point temperature, as shown in <a class="internal-link" href="#table-5">Table 5</a> below. Even small amounts of entrained water (1–3%) in the oil dumped to VRTs or tanks can easily fully saturate the gas, so it's often best practice to assume 100% saturation as a worst-case input, in the absence of water fraction info. However, this can significantly overestimate dew point temperatures, especially in high-temperature applications. 

For this reason, it's a good practice to explicitly request water content in lab gas analyses, as standard reports typically exclude it. 


To reduce this uncertainty, it's good practice to explicitly request lab gas analyses to include water fractions, as standard reports do not include them unless specified. Without this data, engineers are left to estimate saturation levels—sometimes conservatively assuming full saturation, which can lead to overpredicted dew point temperatures. This in turn may falsely eliminate equipment options like screw compressors from consideration due to perceived condensation risk that may not exist under actual conditions.

### 10.1.3. Recommended sampling procedures

### 10.1.4. Process simulation modeling